In [2]:
pip install rouge sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 2.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [3]:
import pandas as pd
import numpy as np
import os
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Input, Embedding, Bidirectional
from keras.optimizers import Adam
from keras.losses import CategoricalCrossentropy
from keras.callbacks import LearningRateScheduler
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge
from sentence_transformers import SentenceTransformer

In [4]:
# Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Function to load data and prepare embeddings
def load_data_and_prepare_embeddings(artist_name):
    file_path = f'/content/drive/MyDrive/Data_processed/processed_LDA_lyrics_with_topic_{artist_name}.csv'
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        model_sbert = SentenceTransformer('all-MiniLM-L6-v2')
        embeddings = model_sbert.encode(df['processed_lyrics'].tolist(), show_progress_bar=True)
        return df, embeddings
    else:
        print("File does not exist.")
        return None, None

In [6]:
# Prepare sequences from text
def prepare_sequences(texts, seq_length=50):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(texts)
    sequences = tokenizer.texts_to_sequences(texts)
    vocab_size = len(tokenizer.word_index) + 1
    dataX, dataY = [], []
    for sequence in sequences:
        for i in range(1, len(sequence) - seq_length):
            seq_in = sequence[i:i + seq_length]
            seq_out = sequence[i + seq_length]
            dataX.append(seq_in)
            dataY.append(seq_out)
    X = pad_sequences(dataX, maxlen=seq_length, padding='pre')
    Y = to_categorical(dataY, num_classes=vocab_size)
    return X, Y, tokenizer, vocab_size

In [7]:
# Create model for text generation
def create_model(seq_length, vocab_size):
    model = Sequential()
    model.add(Embedding(vocab_size, 50, input_length=seq_length))
    model.add(Bidirectional(LSTM(100, return_sequences=True)))
    model.add(Dropout(0.2))
    model.add(Bidirectional(LSTM(100)))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(vocab_size, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [8]:
# Function to train LSTM model
def train_model(X, Y):
    if X.size == 0 or Y.size == 0:
        print("Empty training data: Cannot train model.")
        return None
    seq_length = X.shape[1]
    vocab_size = Y.shape[1]
    model = create_model(seq_length, vocab_size)
    model.fit(X, Y, epochs=100, batch_size=128)
    return model

In [9]:
# Function to generate lyrics
def generate_lyrics(model, tokenizer, seq_length, seed_text, num_words=100):
    result = [seed_text]
    for _ in range(num_words):
        encoded = tokenizer.texts_to_sequences([seed_text])[-1]
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        yhat = model.predict(encoded, verbose=0)
        yhat = np.argmax(yhat, axis=1)
        word = ''
        for key, value in tokenizer.word_index.items():
            if value == yhat:
                word = key
                break
        seed_text += ' ' + word
        result.append(word)
    return ' '.join(result)

In [2]:
# User interaction to select artist and theme
def user_interaction():
    artists = ['21 savage', 'artic monkeys', 'ariana grande', 'billie eilish', 'cardi b',
               'dj khaled', 'drake', 'ed sheeran', 'eminem', 'halsey', 'imagine dragons',
               'justin bieber', 'lady gaga', 'machine gun kelly', 'maroon 5', 'nirvana',
               'pink floyd', 'post malone', 'queen', 'taylor swift', 'the beatles', 'travis scott']
    print("Available artists:")
    for artist in artists:
        print(artist)
    selected_artist = input("Enter an artist from the list: ")

    df, embeddings = load_data_and_prepare_embeddings(selected_artist)
    if df is not None and not df.empty:
        print("Available themes:")
        themes = df['theme'].unique()
        for theme in themes:
            print(theme)
        selected_theme = input("Select a theme from the list: ")

        theme_data = df[df['theme'] == selected_theme]['processed_lyrics'].tolist()
        if not theme_data:
            print("No lyrics data available for the selected theme.")
            return

        X, Y, tokenizer, vocab_size = prepare_sequences(theme_data, seq_length=30)
        if X.size > 0 and Y.size > 0:
            model = train_model(X, Y)
            if model:
                print("Model trained successfully. Please enter a seed phrase to generate lyrics.")
                seed_text = input("Enter a seed phrase of up to 5 words to start the lyrics: ").strip()
                while len(seed_text.split()) > 5:
                    print("Error: Seed phrase must be no more than 5 words.")
                    seed_text = input("Enter a seed phrase of up to 5 words: ").strip()

                generated_lyrics = generate_lyrics(model, tokenizer, 30, seed_text, num_words=100)
                print("Generated Lyrics:\n", generated_lyrics)

                reference_data = theme_data[int(len(theme_data) * 0.8):]
                generated_words = generated_lyrics.split()
                reference_words = [ref.split() for ref in reference_data]

                bleu_score = sentence_bleu(reference_words, generated_words)
                print(f"BLEU Score: {bleu_score}")

                rouge = Rouge()
                rouge_scores = rouge.get_scores(' '.join(generated_words), ' '.join(reference_words[0]))
                print("ROUGE Scores:", rouge_scores)
            else:
                print("Failed to train the model due to an error.")
        else:
            print("Insufficient data to train the model.")
    else:
        print("No data available for the selected artist or theme.")

In [11]:
# Execute the function
user_interaction()

Available artists:
21 savage
artic monkeys
ariana grande
billie eilish
cardi b
dj khaled
drake
ed sheeran
eminem
halsey
imagine dragons
justin bieber
lady gaga
machine gun kelly
maroon 5
nirvana
pink floyd
post malone
queen
taylor swift
the beatles
travis scott
Enter an artist from the list: halsey


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/6 [00:00<?, ?it/s]

Available themes:
Intense Emotion
Bold and Rebellious
Reflection and Depth
Select a theme from the list: Reflection and Depth
Epoch 1/100
93/93 [==============================] - 24s 147ms/step - loss: 5.8963 - accuracy: 0.0336
Epoch 2/100
93/93 [==============================] - 7s 70ms/step - loss: 5.5617 - accuracy: 0.0405
Epoch 3/100
93/93 [==============================] - 4s 42ms/step - loss: 5.3735 - accuracy: 0.0413
Epoch 4/100
93/93 [==============================] - 3s 28ms/step - loss: 5.0066 - accuracy: 0.0625
Epoch 5/100
93/93 [==============================] - 3s 31ms/step - loss: 4.5753 - accuracy: 0.0929
Epoch 6/100
93/93 [==============================] - 2s 21ms/step - loss: 4.2423 - accuracy: 0.1264
Epoch 7/100
93/93 [==============================] - 2s 27ms/step - loss: 3.9603 - accuracy: 0.1700
Epoch 8/100
93/93 [==============================] - 2s 20ms/step - loss: 3.6915 - accuracy: 0.2053
Epoch 9/100
93/93 [==============================] - 2s 17ms/step - loss

In [3]:
user_interaction()

Available artists:
21 savage
artic monkeys
ariana grande
billie eilish
cardi b
dj khaled
drake
ed sheeran
eminem
halsey
imagine dragons
justin bieber
lady gaga
machine gun kelly
maroon 5
nirvana
pink floyd
post malone
queen
taylor swift
the beatles
travis scott
Enter an artist from the list: 21 savage


NameError: name 'load_data_and_prepare_embeddings' is not defined